In [ ]:
# TODO:
# - Geocode
# - Average income
# - Population density
# - Competitors
# - Cost of living impact
# - COVID impact?
# - Footfall?

## Geocode store addresses

In [ ]:
import os
import requests
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv
from shapely.geometry import Point

load_dotenv("../.env")
tomtom_api_key = os.environ.get("TOMTOM_API_KEY")

aldi_df = pd.read_csv("../data/scraped_stores.csv")
aldi_df = aldi_df.iloc[:, 1:]

base_url = "https://api.tomtom.com/search/2/geocode/"

def geocode(address: str) -> Point:
    try:
        res = requests.get(f"{base_url}ALDI, {address}.json?key={tomtom_api_key}&countrySet=GB")
        lat = res.json().get("results", {})[0].get("position", {}).get("lat")
        lon = res.json().get("results", {})[0].get("position", {}).get("lon")
    except Exception:
        return None
    return Point(lon, lat)

aldi_df["geometry"] = aldi_df["full_address"].apply(geocode)
aldi_gdf = gpd.GeoDataFrame(aldi_df, crs="EPSG:4326")
aldi_gdf.to_file("../data/geocoded_stores.geojson", driver="GeoJSON")